# Eat Safe, Love

## Notebook Set Up

In [79]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [80]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [81]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [82]:
# review the collections in our database
pprint(db.establishments.find_one())

{'AddressLine1': '146A Plumstead Rd',
 'AddressLine2': '',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('67a96a230cd9e8814c5eec58'),
 'geocode': {'latitude': 51.490142, 'longitude': 0.08384},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


In [83]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [84]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': 20}

# Use count_documents to display the number of documents in the result
hygiene_documents = establishments.find(query)

# Displprint(f'{establishments.count_documents(query)} total documents')
print(f'{establishments.count_documents(query)} total documents')

0 total documents


In [85]:
# Convert the result to a Pandas DataFrame
hygiene_df = pd.DataFrame(hygiene_documents)

# Display the number of rows in the DataFrame
print(f'{len(hygiene_df)} total rows')

# Display the first 10 rows of the DataFrame
hygiene_df.head(10)

0 total rows


""


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [86]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {
    '$and': [
        {'LocalAuthorityName': {'$regex': 'london', '$options': 'i'}},
        {'RatingValue': {'$gte': 4}}
    ]
}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"Number of establishments in London with RatingValue >= 4: {count}")

# Display the first document in the results using pprint
result = establishments.find_one(query)
if result:
    pprint(result)
else:
    pass

Number of establishments in London with RatingValue >= 4: 0


In [87]:
# Convert the result to a Pandas DataFrame
london_ratings_df = pd.DataFrame(list(establishments.find(query)))

# Display the number of rows in the DataFrame
print(f"Number of rows: {len(london_ratings_df)}")

# Display the first 10 rows of the DataFrame
if not london_ratings_df.empty:
    print(f"Number of rows: {len(london_ratings_df)}")
    london_ratings_df.head(10)
else:
    pass

Number of rows: 0


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [88]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

query = {
    "RatingValue": 5,
    "geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search},
}
sort = [("scores.Hygiene", 1)]
limit = 5

# Print the results
results = list(establishments.find(query).sort(sort).limit(limit))

if results:
    pprint(results)
else:
    print("No documents found for the query.")

No documents found for the query.


In [89]:
# Convert result to Pandas DataFrame
top_5_df = pd.DataFrame(results)
if not top_5_df.empty:
    top_5_df.head(10)
else:
    print("No establishments found in London with RatingValue >= 4.")

No establishments found in London with RatingValue >= 4.


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [90]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

match_query = {'$match': {'scores.Hygiene': 0}}
group_query = {'$group': {'_id': '$LocalAuthorityName', 'count': {'$sum': 1}}}
sort_query = {'$sort': {'count': -1}}

pipeline = [match_query, group_query, sort_query]

# Print the number of documents in the result
results = list(establishments.aggregate(pipeline))

# Print the first 10 results
if results:
    pprint(results[:10])
else:
    print("No establishments with a hygiene score of 0.")

No establishments with a hygiene score of 0.


In [91]:
# Convert the result to a Pandas DataFrame
hygiene_df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
num_rows_df = pd.DataFrame({'Number of Rows': [hygiene_df.shape[0]]})
display(num_rows_df)

# Display the first 10 rows of the DataFrame
if not hygiene_df.empty:
    print(f"Number of rows: {len(hygiene_df)}")
    hygiene_df.head(10)
else:
    print("No establishments found with the specified hygiene score.")

,Number of Rows
0,0


No establishments found with the specified hygiene score.
